In [1]:
import matplotlib.pyplot as plt
from scipy.stats import norm
import numpy as np
import scipy.stats as stats
import pylab as pl
from sklearn import metrics  
import pandas as pd
from scipy.stats import linregress
from itertools import combinations 
import pandasql as psql
import glob
import os
import fnmatch
import scipy.spatial.distance as dist
import scipy.cluster.hierarchy as hac
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.optimize import minimize, rosen, rosen_der
from joblib import Parallel, delayed
import multiprocessing

In [2]:
#path = "C:/Users/bennettf/Desktop/New folder (7)/APSIMsims/"
#path = "D:/Burdekin/apsim_stuff/APSIMsims/"
#path = "D:/Burdekin/apsim_stuff/corr/csvs/"
path = "R:/ReefSugar_RC7/BU/BU_RatoonAggregated_NoTotals/"
#path = "R:/ReefSugar_RC7/BU/smallSet/"
outpath = "R:/ReefSugar_RC7/BU/clusters_din/"
#column_name = "Nleached"
#column_names = ['Runoff','soil_loss2','Grants_DINrunoff','Nleached']
column_names = ['Grants_DINrunoff','Nleached']
extens = '.csv'

In [5]:
#data_set = pd.DataFrame()

dataframe_collection = {}
encounteredCombos = []

num_cores = multiprocessing.cpu_count()
    
file_list = fnmatch.filter(os.listdir(path), '*' + extens)

#Parallel(n_jobs=num_cores)(for file in file_list):
for file in file_list :
    # compose a column heading name from the filename
    #file_split = file.split('_')
    #coom$1955_14710_adj$bpafaf$irra.csv
    #print('Processing file: ' + file)
    file_split = file.replace(extens, '').split('$')
    soil = file_split[0]
    climate = file_split[1]
    #ks = file_split[7].split(' ')[0]
    mgt1 = file_split[2]
    irrig = file_split[3]
    mgt = mgt1 + '_' + irrig
    #baseCombo = soil + '_' + climate + '_' + irrig
    baseCombo = soil + '_' + climate
    #heading = mgt
    
    if baseCombo not in encounteredCombos:
        encounteredCombos.append(baseCombo)
        print('Adding combo: ' + baseCombo)
        dataframe_collection[baseCombo] = {}
        
        for colName in column_names:
            dataframe_collection[baseCombo][colName] = pd.DataFrame()
    
    file_in = pd.read_csv(path + file, skiprows=[1])
    #print('Reading file: ' + file)
    
    for colName in column_names:
        dataframe_collection[baseCombo][colName][mgt] = file_in[colName]
    
    


Adding combo: coom_1955_14710_adj
Adding combo: coom_1970_14735_adj
Adding combo: coom_1975_14710_adj
Adding combo: coom_2010_14725_adj
Adding combo: garb_1955_14710_adj
Adding combo: garb_1970_14735_adj
Adding combo: garb_1975_14710_adj
Adding combo: garb_2005_14815_adj
Adding combo: garb_2010_14725_adj
Adding combo: hatc_1955_14710_adj
Adding combo: hatc_1970_14735_adj
Adding combo: hatc_1975_14710_adj
Adding combo: hatc_2005_14815_adj
Adding combo: hatc_2010_14725_adj
Adding combo: kala_1955_14710_adj
Adding combo: kala_1970_14735_adj
Adding combo: kala_1975_14710_adj
Adding combo: mari_1955_14710_adj
Adding combo: mari_1970_14735_adj
Adding combo: mari_1975_14710_adj
Adding combo: mari_2005_14815_adj
Adding combo: mari_2010_14725_adj
Adding combo: neil_1955_14710_adj
Adding combo: neil_1970_14735_adj
Adding combo: neil_1975_14710_adj
Adding combo: neil_2010_14725_adj


In [6]:

for baseCombo in encounteredCombos:
    #here
    results = pd.DataFrame()
    
    counter = 1
    for colName in column_names:
        #here
        data_matrix = dataframe_collection[baseCombo][colName].T.as_matrix()
        distance_matrix = dist.pdist(data_matrix, 'euclidean' )
        Z = hac.linkage(distance_matrix, method='ward', metric='euclidean')
        T = hac.fcluster(Z, 12, criterion='maxclust')
        
        if counter == 1:
            #grab the mgt codes as a column
            results['mgt'] = list(dataframe_collection[baseCombo][colName].columns.values)
        
        results[colName] = T
        counter += 1
        
    #write out stuff
    results.to_csv(outpath + baseCombo + '_MaxClust12.csv')



#data_matrix = data_set.T.as_matrix()
##distance_matrix = dist.squareform(dist.pdist(data_matrix, 'euclidean' ))
#distance_matrix = dist.pdist(data_matrix, 'euclidean' )
#Z = hac.linkage(distance_matrix, method='ward', metric='euclidean')
#fig = plt.figure(figsize=(25, 10))
#plt.title('Hierarchical Clustering Dendrogram')
#plt.xlabel('sample index')
#plt.ylabel('distance')
#dn = dendrogram(Z)
#plt.show()

In [6]:
print('done')

done
